# Advent of Code 2024, Day 6

In [ ]:
with open('input.txt', 'r') as f:
    puzzle_input = f.read().strip()

## [First Puzzle:](https://adventofcode.com/2024/day/6)

In [ ]:
import numpy as np
from itertools import cycle, islice

In [ ]:
matrix = np.array([list(line) for line in puzzle_input.split('\n')])
guard_moves = {
    '^': (-1, 0),
    '>': (0, 1),
    'v': (1, 0),
    '<': (0, -1)
}
guard_move_keys = list(guard_moves.keys())

In [ ]:
np_coords = np.where(np.isin(matrix, guard_move_keys))
if np_coords[0].size == 0:
    raise ValueError('No guard found')

guard_coords = tuple(c.item() for c in np_coords)
guard_char = matrix[guard_coords]

In [ ]:
start_index = guard_move_keys.index(guard_char)
cyclic_keys = cycle(guard_move_keys[start_index:] + guard_move_keys[
                                                    :start_index])  # Hilariously complicated way to set a starting index
next(cyclic_keys)  # Skip the first key

In [ ]:
def is_valid_index(array, index: tuple):
    index = np.array(index)
    return (index >= 0).all() and (index < array.shape).all()

In [ ]:
def get_next_guard(matrix, coords, direction):
    next_coords = tuple(np.add(coords, guard_moves[direction]))
    if not is_valid_index(matrix, next_coords):
        return (None, None)

    next_char = matrix[next_coords]

    while next_char == '#':
        direction = next(cyclic_keys)
        next_coords = tuple(np.add(coords, guard_moves[direction]))
        next_char = matrix[next_coords]

        if not is_valid_index(matrix, next_coords):
            return (None, None)

    return next_coords, direction

In [ ]:
old_coords = guard_coords
current_coords = guard_coords

In [ ]:
iterations = 0
while current_coords:
    iterations += 1
    matrix[current_coords] = guard_char
    matrix[old_coords] = 'X'
    old_coords = current_coords
    current_coords, guard_char = get_next_guard(matrix, current_coords, guard_char)

matrix[old_coords] = 'X'

In [ ]:
np.count_nonzero(matrix == 'X')

## [Second Puzzle:](https://adventofcode.com/2024/day/6/#part2)

In [ ]:
# puzzle_input = """....#.....
# .........#
# ..........
# ..#.......
# .......#..
# ..........
# .#..^.....
# ........#.
# #.........
# ......#..."""

In [ ]:
matrix = np.array([list(line) for line in puzzle_input.split('\n')])
guard_moves = {
    '^': (-1, 0),
    '>': (0, 1),
    'v': (1, 0),
    '<': (0, -1)
}
guard_cycle = {
    '^': '>',
    '>': 'v',
    'v': '<',
    '<': '^'
}

In [ ]:
np_coords = np.where(np.isin(matrix, guard_move_keys))
if np_coords[0].size == 0:
    raise ValueError('No guard found')

guard_coords = tuple(c.item() for c in np_coords)
guard_char = matrix[guard_coords]

In [ ]:
def get_next_coords(matrix, current_coords, current_direction):
    next_direction = current_direction
    next_coords = tuple(np.add(current_coords, guard_moves[current_direction]))

    if not is_valid_index(matrix, next_coords):
        return (None, None)

    next_char = matrix[next_coords]

    while next_char == '#':
        next_direction = guard_cycle[next_direction]
        next_coords = tuple(np.add(current_coords, guard_moves[next_direction]))

        if not is_valid_index(matrix, next_coords):
            return (None, None)

        next_char = matrix[next_coords]

    return next_coords, next_direction

In [ ]:
def is_cycle(matrix, current_coords, current_direction) -> bool: # Returns True if the guard is stuck in a loop
    steps = 0
    while current_coords:
        current_coords, current_direction = get_next_coords(matrix, current_coords, current_direction)
        steps += 1
        if steps > 2*matrix.size:
            return True
    return False

In [ ]:
rows, cols = matrix.shape
new_matrices = []

for r in range(rows):
    for c in range(cols):
        blockade_coords = (r,c)
        if blockade_coords != guard_coords and matrix[r, c] != "#":
            new_matrix = matrix.copy()
            new_matrix[r, c] = "#"
            new_matrices.append((blockade_coords, new_matrix))

In [ ]:
from tqdm import tqdm

In [ ]:
blockades = set()
for blockade_coords, new_matrix in tqdm(new_matrices):
    if is_cycle(new_matrix, guard_coords, guard_char):
        blockades.add(blockade_coords)

In [ ]:
len(blockades) # 1328